###    Pipeline to illustrate GWAS ina tetraploid species
    It uses gen format instead of vcf. gen format contains
       chr pos alleles ....
    space / tab separated


In [ ]:
import numpy as np
import gzip
import sys
import time
import os
import matplotlib.pyplot as plt

# specific seqbreed modules
import genome as gg
import selection as sel

In [ ]:
# current dir
cdir = os.getcwd()

# working directory
wdir = cdir

# input file dir
ddir = cdir

# working files, seqfile contains snp positions from vcf
# this file is generated by gg.GFounder below
seqfile = 'seq.pos'

# list of chips that can be analyzed, include sequence as output from gg.GFounder()
chipfiles = [seqfile]
# contains qtn positions
qtnfile = ddir + '/potato.qtn.pos'
# gene dropping pedigree
pedfile = ddir + '/potato.ped'
# snp file in gen format
genfile = ddir + '/potato.gen.gz'

# goto working directory
os.chdir(wdir)



In [ ]:
# STEP 1:
# uploads genotypes and generates snp positions (snpFile)
# NOTE that ploidy level must be specified if gen format
gbase = gg.GFounder(vcfFile=genfile, snpFile=seqfile, ploidy=4)

# STEP 2: generates Genome object with chr names, recombination map, etc
gfeatures = gg.Genome(snpFile=seqfile, ploidy=gbase.ploidy)

# prints some basic info
gfeatures.print()

# STEP 3: read QTN file
qtn = gg.QTNs(h2=[0.5, 0.7], genome=gfeatures, qtnFile=qtnfile)
qtn.get_var(gfeatures,gbase)

# STEP 4: generate base population
pop = gg.Population(gfeatures, pedFile=None, generation=None, qtns=qtn, gfounders=gbase)
qtn.print(gfeatures)



In [ ]:
# STEP 5: Simulate a F2 between the 10 most extreme lowest and highest lines for second phenotype
#--> Sort phenotypes and sel parents
itrait = 1
nsel = 10
y = np.array(list(pop.inds[i].y[itrait] for i in range(pop.n))) # phenotypes
yindex = np.argsort(y)
ilow = yindex[:nsel]
ihigh = np.random.permutation(yindex[-nsel:]) # random mating between extremes

#--> F1 between lowest and highest inds
n0 = pop.n
for i in range(nsel):
    parents = [pop.inds[ilow[i]], pop.inds[ihigh[i]] ]
    pop.addInd(parents, genome=gfeatures, gfounders=gbase, qtns=qtn, id=None, sex=None, t=1)

#--> F2 by selfcrossing
noff = 20
n1 = pop.n
for i in range(nsel):
    iid = n0+i # index of parent
    parents = [pop.inds[iid], pop.inds[iid]]
    for j in range(noff):
        pop.addInd(parents, genome=gfeatures, gfounders=gbase, qtns=qtn, id=None, sex=None, t=2)

# STEP 5: GWAS
# generate chip objects
chipseq = gg.Chip(chipFile=seqfile, genome=gfeatures, name='seq_chip')

# get genotypes
X = gg.do_X(pop.inds, gfeatures, gbase, chip=chipseq)

# GWAS for second trait
gwas = sel.Gwas(X, chipseq)
gwas.fit(pop.inds, trait=1)
gwas.plot(fdr=True)    # FDR

In [ ]:
# GWAS for first trait
gwas = sel.Gwas(X, chipseq)
gwas.fit(pop.inds, trait=0)
gwas.plot(fdr=True)    # FDR
gwas.print(gfeatures)  # prints gwas results

In [ ]:
# Appendix: How to simulate repeated measurements
itrait = 0
current_y = np.array(list(ind.y[itrait] for ind in pop.inds))
new_y = np.array([])
for ind in pop.inds:
    y = ind.g_add[itrait] + ind.g_dom[itrait] + np.random.normal(0, qtn.se[itrait])
    new_y = np.append(new_y, y)

# plot new_y vs current_y
plt.scatter(new_y, current_y)
plt.title('Repeated measurements: New vs Old phenotypes')
plt.xlabel('New')
plt.ylabel('Old')
plt.show()
plt.close()